# Gen AI Project 

In [1]:
# ! pip install spacy
# !python -m spacy download en_core_web_lg
# !python -m spacy download en_core_web_lg 


In [2]:
import json
import pandas as pd
import re
import openai
import os
import ast
import time

from openai import OpenAI
client = OpenAI()
# similarity score
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np


openai.api_key = os.getenv("OPENAI_API_KEY")

In [3]:
# Open the file and load the JSON content
file_path = 'src/input/Dataset for DS Case Study.json'

# Open and read the file
with open(file_path, 'r') as file:
    data = [json.loads(line) for line in file]

# Output the first entry for verification
print(data[0])  # Displays the first review



{'reviewerID': 'A3F73SC1LY51OO', 'asin': 'B00002243X', 'reviewerName': 'Alan Montgomery', 'helpful': [4, 4], 'reviewText': "I needed a set of jumper cables for my new car and these had good reviews and were at a good price.  They have been used a few times already and do what they are supposed to - no complaints there.What I will say is that 12 feet really isn't an ideal length.  Sure, if you pull up front bumper to front bumper they are plenty long, but a lot of times you will be beside another car or can't get really close.  Because of this, I would recommend something a little longer than 12'.Great brand - get 16' version though.", 'overall': 5.0, 'summary': 'Work Well - Should Have Bought Longer Ones', 'unixReviewTime': 1313539200, 'reviewTime': '08 17, 2011'}


In [4]:
df =pd.DataFrame(data)
df.head(5)

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A3F73SC1LY51OO,B00002243X,Alan Montgomery,"[4, 4]",I needed a set of jumper cables for my new car...,5.0,Work Well - Should Have Bought Longer Ones,1313539200,"08 17, 2011"
1,A20S66SKYXULG2,B00002243X,alphonse,"[1, 1]","These long cables work fine for my truck, but ...",4.0,Okay long cables,1315094400,"09 4, 2011"
2,A2I8LFSN2IS5EO,B00002243X,Chris,"[0, 0]",Can't comment much on these since they have no...,5.0,Looks and feels heavy Duty,1374710400,"07 25, 2013"
3,A3GT2EWQSO45ZG,B00002243X,DeusEx,"[19, 19]",I absolutley love Amazon!!! For the price of ...,5.0,Excellent choice for Jumper Cables!!!,1292889600,"12 21, 2010"
4,A3ESWJPAVRPWB4,B00002243X,E. Hernandez,"[0, 0]",I purchased the 12' feet long cable set and th...,5.0,"Excellent, High Quality Starter Cables",1341360000,"07 4, 2012"


In [5]:
df.shape

(100, 9)

In [6]:
# Fill NaN values with empty strings
df['reviewText'] = df['reviewText'].fillna('')

# Function to clean text
def clean_text(text):
    text = text.lower()  # Lowercase
    text = re.sub(r'\s+', ' ', text)  # Remove extra spaces
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    return text

# Apply cleaning
df['cleaned_reviewText'] = df['reviewText'].apply(clean_text)

df[['reviewText', 'cleaned_reviewText']].head(19)


labels = [
    "person",      # people, including fictional characters
    "fac",         # buildings, airports, highways, bridges
    "org",         # organizations, companies, agencies, institutions
    "gpe",         # geopolitical entities like countries, cities, states
    "loc",         # non-gpe locations
    "product",     # vehicles, foods, appareal, appliances, software, toys 
    "event",       # named sports, scientific milestones, historical events
    "work_of_art", # titles of books, songs, movies
    "law",         # named laws, acts, or legislations
    "language",    # any named language
    "date",        # absolute or relative dates or periods
    "time",        # time units smaller than a day
    "percent",     # percentage (e.g., "twenty percent", "18%")
    "money",       # monetary values, including unit
    "quantity",    # measurements, e.g., weight or distance
    "automobile",  # name (e.g., brand or model names)
    "Components",   #(e.g., engine, tires, steering wheel, etc.)
]

In [41]:
def ner(free_text): 
     # Define a prompt to instruct the model to extract entities
    prompt = f"""
    Extract all possible named entities from the following text:

    Text: \"{free_text}\"

    Entities should include people, organizations, locations, products, and any other relevant categories.
    Return the entities in a JSON format with their category (e.g., "person", "organization", "location", "product").
    """

    # Make the API call to extract entities
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "Extract all possible named entities from the following text:"},
            {"role": "user", "content": f"Entities should include {labels}, and any other relevant categories. "},
            {"role": "assistant", "content": """Return only the entities in a JSON format with their category (e.g., "person", "organization", "location", "product"). no extra text , connects of not available keep it null"""},
            {"role": "user", "content": f"{free_text}"}
  ]
       
    )

    # Get the model's response
    entities = response.choices[0].message.content
    time.sleep(2)

    return entities


def clean_json(text):
    try:
        print(json.loads(text[8:-4]))
        return  json.loads(text[8:-4])
    except:
        print(json.loads(text[:]))
        return  json.loads(text[:])

# ner_tagging = pd.json_normalize(df['JSON_clean'])

def get_embedding(text, model="text-embedding-3-small"):
    response = client.embeddings.create(
        input=text,
        model=model
    )
    return response.data[0].embedding

def recommend_product(review_text,df,top_n):
    input_embedding = get_embedding (review_text)
    similarities = cosine_similarity([input_embedding], list(df['Embeddings']))
    top_indices = np.argsort(similarities[0][:top_n])
    return df.iloc[top_indices]

def Recommendation(text): 
     # Define a prompt to instruct the model to extract entities
    prompt = f"""{text}"""
    # Make the API call to extract entities
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "You are helpgul assistant to extract insight from json structure"},
            {"role": "user", "content": "Text is given in below statement in json format extract only product name only "},
            {"role": "assistant", "content": " statement in json format extract information and restruct in english sentence,only englist statement no additional comments "},
            {"role": "user", "content": f"{text}"}
             ],
            temperature= 0
            )

    # Get the model's response
    entities = response.choices[0].message.content
    time.sleep(2)

    return entities


In [19]:
input_embedding = get_embedding("I am planning to buy insurance for my car, can you recommend some best car insurance company?")

similarities = cosine_similarity([input_embedding], list(df['Embeddings']))

similarities[0]

df['NER_Tagging'] = df['reviewText'].apply(ner)

df['JSON_clean'] = df['NER_Tagging'].apply(clean_json)
# merge_pd = pd.merge(df,ner_tagging,left_index=True, right_index=True)

df['Embeddings'] = df['reviewText'].apply(get_embedding)

### Ask Question based upon review Text

In [42]:
text = "I am planning to buy insurance for my car, can you recommend some best car insurance company?"
# text = input()
top_n =5  # top 5 recommendations
l = set()
for i in recommend_product(text,df,top_n=5)['JSON_clean']:
    l.add(Recommendation(i))
    time.sleep(2)

print(l)

{'Booster/Jumper Cables', 'bucket boss 06009 jumper cable bag', 'jumper cables', 'The product name is "cable set".', 'truck'}


# Finetuning

In [3]:
import openai 
import json
import os
import pandas as pd
from openai import OpenAI
client = OpenAI()
os.getenv("OPENAI_API_KEY")
print('')

In [4]:
# uploading training file, using 50 datapoints to train model

file_path = 'src/output/50_items_for_finetuning.jsonl'
response_id = client.files.create(
  file=open(file_path, "rb"),
  purpose="fine-tune"
)


In [5]:
response_id

FileObject(id='file-WCiTP1x3KNn5aRgr8MyqB3zP', bytes=15524, created_at=1727726152, filename='50_items_for_finetuning.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [6]:
client.fine_tuning.jobs.create(
  training_file="file-WCiTP1x3KNn5aRgr8MyqB3zP", 
  model="gpt-4o-mini-2024-07-18"
)

FineTuningJob(id='ftjob-jMimL1tIz6hsBFxuYRseRJdM', created_at=1727726182, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-EArhIgizr0y7BT33QNhx4AnQ', result_files=[], seed=788730453, status='validating_files', trained_tokens=None, training_file='file-WCiTP1x3KNn5aRgr8MyqB3zP', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)

In [ ]:
completion = client.chat.completions.create(
  model="ft:gpt-4o-mini-2024-07-18:personal:finetune:ADGIDfMw",
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Can you recommend some product for car" }
  ]
)
print(completion.choices[0].message.content)